<a href="https://colab.research.google.com/github/Vignesh123k/CRACKING-THE-MARKET-CODE-WITH-AI-DRIVEN-/blob/main/phase_2_nm303.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Load dataset (example: from a CSV or Yahoo Finance API)
# df = pd.read_csv('AAPL.csv')  # Uncomment if reading from CSV
# OR use yfinance API:
import yfinance as yf
df = yf.download('AAPL', start='2010-01-01', end='2025-01-01')

# 1. Set 'Date' as index
df.index = pd.to_datetime(df.index)

# 2. Handle missing values
df.fillna(method='ffill', inplace=True)

# 3. Plot Closing Price
df['Close'].plot(figsize=(12, 5), title="AAPL Closing Price Over Time")
plt.xlabel("Date")
plt.ylabel("Close Price (USD)")
plt.show()

# 4. Feature Engineering
df['Close_lag1'] = df['Close'].shift(1)
df['MA_7'] = df['Close'].rolling(window=7).mean()
df['MA_30'] = df['Close'].rolling(window=30).mean()
df['STD_7'] = df['Close'].rolling(window=7).std()

# Log returns
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))

# Relative Strength Index (RSI)
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

df['RSI'] = compute_rsi(df['Close'])

# Drop rows with NaNs from rolling features
df.dropna(inplace=True)

# 5. Normalize Features for LSTM (only on relevant features)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close', 'Close_lag1', 'MA_7', 'MA_30', 'STD_7', 'RSI']])

# Convert to DataFrame
scaled_df = pd.DataFrame(scaled_data, index=df.index, columns=['Close', 'Close_lag1', 'MA_7', 'MA_30', 'STD_7', 'RSI'])

# Final check
print(scaled_df.head())
